In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tweepy
from tweepy import OAuthHandler
import csv
import cv2
import numpy as np
import sys
import random
import pandas as pd
import math
import glob
import os
import io
import re
import string
import operator
import itertools
from collections import Counter
from collections import defaultdict 
import json
import nltk
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI
from nltk.tokenize import word_tokenize
import urllib3
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
def rnn(train_X,train_Y,test_X,test_Y):
    top_words = 50000
    max_review_length = 250
    train_X = sequence.pad_sequences(train_X, maxlen=max_review_length)
    test_X = sequence.pad_sequences(test_X, maxlen=max_review_length)
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(train_X, train_Y, validation_data=(test_X, test_Y), nb_epoch=3, batch_size=64)
    
    scores = model.evaluate(test_X, test_Y, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))

In [3]:
def load_data(per_train,f1name,f2name):
    data1 = pd.read_csv(f1name)
    data2 = pd.read_csv(f2name)
    size1 = (len(data1.index)-1)*per_train
    size1/=100
    size2 = (len(data2.index)-1)*per_train
    size2/=100
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []
    for i in range(0,size1):
        temp = data1.iloc[i,3].split()
        train_X.append(temp)
        train_Y.append(data1.iloc[i,4])
    
    for i in range(size1,len(data1.index)):
        temp = data1.iloc[i,3].split()
        test_X.append(temp)
        test_Y.append(data1.iloc[i,4])
        
    for i in range(0,size2):
        temp = data2.iloc[i,3].split()
        train_X.append(temp)
        train_Y.append(data2.iloc[i,4])
        
    for i in range(size2,len(data2.index)):
        temp = data2.iloc[i,3].split()
        test_X.append(temp)
        test_Y.append(data2.iloc[i,4])
        
    #print train_X
    #print train_Y
    
    trainX = np.array(train_X)
    trainY = np.array(train_Y)
    testX = np.array(test_X)
    testY = np.array(test_Y)
    
    rnn(trainX,trainY,testX,testY)

In [4]:
if __name__ == '__main__':
    per_train = 30
    f1name = "Clinton_nn_data.csv"
    f2name = "Trump_nn_data.csv"
    while(per_train<=30):
        load_data(per_train,f1name,f2name)
        per_train+=10
    

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 250, 32)       1600000     embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1)             101         lstm_1[0][0]                     
Total params: 1653301
____________________________________________________________________________________________________
None
Train on 8282 samples, validate on 19328 samples
Epoch 1/3
8282/8282 [==============================] - 162s - loss: 0.6039 - acc: 0.6659 - val_loss: 0.6238 - val_ac